In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sm

from utils import timeit
from forcefield import LennardJonesPotential

The Jacobi of the dipole function has the form:
\begin{align*}
    \begin{bmatrix}
    \partial \mu_x / \partial r_x & \partial \mu_y / \partial r_x & \partial \mu_z / \partial r_x 
    \\
    \partial \mu_x / \partial r_y & \partial \mu_y / \partial r_y & \partial \mu_z / \partial r_y 
    \\
    \partial \mu_x / \partial r_z & \partial \mu_y / \partial r_z & \partial \mu_z / \partial r_z
    \end{bmatrix}
\end{align*}

In [3]:
sm_Rax = sm.symbols("Rax")
sm_Ray = sm.symbols("Ray")
sm_Raz = sm.symbols("Raz")
sm_Ra = [sm_Rax, sm_Ray, sm_Raz]

sm_Rbx = sm.symbols("Rbx")
sm_Rby = sm.symbols("Rby")
sm_Rbz = sm.symbols("Rbz")
sm_Rb = [sm_Rbx, sm_Rby, sm_Rbz]

sm_mu0 = sm.symbols("mu0")
sm_a = sm.symbols("a")
sm_d0 = sm.symbols("d0")
sm_d1 = sm.symbols("d1")
sm_d2 = sm.symbols("d2")
sm_d7 = sm.symbols("d7")

class BaseDipoleFunction:
    def __init__(self):
        
        self.dipole_function = [
            sm.lambdify(
            [sm_Rax, sm_Ray, sm_Raz, sm_Rbx, sm_Rby, sm_Rbz, 
             ] + self.parameters_sm, 
                dipole_exp
        ) for dipole_exp in self.dipole_exp]
        
        self.generate_Jacobi()
        #self.generate_Hessian()

    def generate_Jacobi(self):
        self.Ja_exp = [[],[],[]]
        self.Ja_func = [[],[],[]]

        self.Jb_exp = [[],[],[]]
        self.Jb_func = [[],[],[]]

        for i,Ra in enumerate(sm_Ra):
            for j,dipole_f in enumerate(self.dipole_exp):
                d_mu = sm.diff(dipole_f, Ra)
                self.Ja_exp[i].append(d_mu)
                self.Ja_func[i].append(sm.lambdify(
                    [sm_Rax, sm_Ray, sm_Raz, sm_Rbx, sm_Rby, sm_Rbz, 
                    ]+ self.parameters_sm, d_mu
                ))

                d_mu = sm.diff(dipole_f, sm_Rb[i])
                self.Jb_exp[i].append(d_mu)
                self.Jb_func[i].append(sm.lambdify(
                    [sm_Rax, sm_Ray, sm_Raz, sm_Rbx, sm_Rby, sm_Rbz, 
                    ]+ self.parameters_sm, d_mu
                ))

    def __call__(self, ra, rb):
        """
        Note: 
        ra should be the positive particle and 
        rb should be the negative particle
        """
        ra = list(ra)
        rb = list(rb)

        args = ra + rb + self.parameters

        result = [dipole_f(*args) for dipole_f in self.dipole_function]
        
        return result

    def gradient(self,ra,rb,sign = "+"):
        ra = list(ra)
        rb = list(rb)
        args = ra + rb + self.parameters
        result = np.zeros((3,3))
        
        if sign == "+": 
            J_func = self.Ja_func
        elif sign == "-": 
            J_func = self.Jb_func
            
        for i, dmu_dRa in enumerate(J_func):
            for j, dmu_i_dRa in enumerate(dmu_dRa):
                result[i,j] = dmu_i_dRa(*args)

        return result

class GriegorievDipoleFunction(BaseDipoleFunction):
    def __init__(self, mu0, a, d0, d7):

        sm_d = ((sm_Rax - sm_Rbx)**2 + (sm_Ray - sm_Rby)**2 \
             + (sm_Raz - sm_Rbz)**2)**(1/2)
        
        self.parameters = [mu0, a, d0, d7]
        self.parameters_sm = [sm_mu0, sm_a, sm_d0, sm_d7]
        self.dipole_exp = [
            ((sm_Ra[i] - sm_Rb[i])/sm_d) *\
            (sm_mu0 * sm.exp(-sm_a*(sm_d-sm_d0)) - sm_d7/(sm_d**7))
            for i in range(3)
        ]
        
        super().__init__()

dipole_function1 = GriegorievDipoleFunction(mu0=0.0284, a=1.22522, d0=7.10, d7=14200)

print(dipole_function1([0,0,0],[5,0,0]))
print(dipole_function1.gradient([0,0,0],[5,0,0],"+"))
print(dipole_function1.gradient([5,0,0],[0,0,0],"-"))

[-0.19041015111388537, 0.0, 0.0]
[[-0.20152631  0.          0.        ]
 [ 0.          0.03808203  0.        ]
 [ 0.          0.          0.03808203]]
[[ 0.20152631  0.          0.        ]
 [ 0.         -0.03808203  0.        ]
 [ 0.          0.         -0.03808203]]


In [4]:
def dot_C(r, C, jk):
    """
    Computing partial derivative of C w.r.t. time, a.k.a. C_dot
    Args:
    + r (np.array): list/array of postition of charged particles. Shape: n_particles x 3
    + v (np.array): list/array of velocities of charged particles. Shape: n_particles x 3
    + C (np.array): list/array of pair of modes
    """

    C_dot = []

    jk_transv = (np.eye(3) - np.outer(k_vec, k_vec) / (self.k[j]**2)) @ jk

    proj_jk_transv = np.array([
        jk_transv @ e for e in self.epsilon[j] 
        ])

    C_dot.append( -1j * self.omega[j] * C[j] + \
        (2 * np.pi * 1j / self.k[j]) * proj_jk_transv)

    C_dot = np.array(C_dot)
    return C_dot


In [6]:
r_ar = np.array([2,2,2]) # Argon should be negative
r_xe = np.array([0,0,0]) # Xenon should be positive

v_ar = np.array([-1,-1,-1])
v_xe = np.array([0,0,0])

k_vec = np.array([1,0,0])
C = np.tile((np.random.rand(1,2) + 1j * np.random.rand(1,2)),(1,1)),

dipole_func = GriegorievDipoleFunction(mu0=0.0284, a=1.22522, d0=7.10, d7=14200)
mu = dipole_func(r_xe, r_ar)

mu_grad_ar = dipole_func.gradient(r_ar, r_xe, "-")

j_mu = mu_grad_ar.T @ r_ar
jk_mu = j_mu * np.exp(-1j * k_vec @ r_ar)

In [23]:
j_mu

array([3.59967386, 3.59967386, 3.59967386])

In [32]:
dipole_func.gradient(r_ar, r_xe) - dipole_func.gradient(r_xe, r_ar)

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [31]:
dipole_func.gradient(r_xe, r_ar)

array([[0.6136541 , 0.59309142, 0.59309142],
       [0.59309142, 0.6136541 , 0.59309142],
       [0.59309142, 0.59309142, 0.6136541 ]])